# Arxiv Search with OpenCLIP and LanceDB

In this example we'll build a Arxiv Search or a recommender based on semantic search using LanceDB. We'll also compare the results with keyword based search on Nomic's atlast


## OpenCLIP

![CLIP (1)](https://github.com/lancedb/vectordb-recipes/assets/15766192/11b3b900-0bcb-4a4a-8fd4-804611c85972)


OpenCLIP an open source implementation of OpenAI's CLIP (Contrastive Language-Image Pre-training) as is available with various backends

In [1]:
# SETUP
!pip install lancedb open_clip_torch arxiv --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 107.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 9.2 MB/s eta 0:00:00


In [2]:
!pip install pandas

## Creating table from arxiv API

### Embedding Paper Summary using CLIP


In [3]:
import torch
import open_clip
import pandas as pd
from open_clip import tokenizer
from tqdm import tqdm
from collections import defaultdict
import arxiv
import lancedb


def embed_func_clip(text):
    model, _, preprocess = open_clip.create_model_and_transforms(
        "ViT-B-32", pretrained="laion2b_s34b_b79k"
    )
    tokenizer = open_clip.get_tokenizer("ViT-B-32")
    with torch.no_grad():
        text_features = model.encode_text(tokenizer(text))
    return text_features

### Create a DataFrame of the desired length

Here we'll use arxiv python utility to interact with arxiv api and get the document data

In [4]:
def get_arxiv_df(embed_func, length=100):
    results = arxiv.Search(
        query="cat:cs.AI OR cat:cs.CV OR cat:stat.ML",
        max_results=length,
        sort_by=arxiv.SortCriterion.Relevance,
        sort_order=arxiv.SortOrder.Descending,
    ).results()
    df = defaultdict(list)
    for result in tqdm(results, total=length):
        try:
            df["title"].append(result.title)
            df["summary"].append(result.summary)
            df["authors"].append(str(result.authors))
            df["url"].append(result.entry_id)
            df["vector"].append(embed_func(result.summary).tolist()[0])

        except Exception as e:
            print("error: ", e)

    return pd.DataFrame(df)

In [5]:
LENGTH = 100  # Reduce the size for demo


def create_table():
    db = lancedb.connect("db")
    df = get_arxiv_df(embed_func_clip, LENGTH)

    tbl = db.create_table("arxiv", data=df, mode="overwrite")

    return tbl

In [6]:
tbl = create_table()

<ipython-input-4-32cd16380017>:7: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  ).results()
  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

100%|██████████| 100/100 [02:57<00:00,  1.77s/it]


In [7]:
import lancedb

db = lancedb.connect("db")

if "arxiv" not in db.table_names():
    tbl = create_table()
else:
    tbl = db.open_table("arxiv")

In [8]:
len(tbl)

100

## Semantic Search by concepts or summary

In [9]:
from IPython.display import display, HTML


def search_table(query, embed_func=embed_func_clip, lim=3):
    db = lancedb.connect("db")
    tbl = db.open_table("arxiv")

    embs = embed_func(query)

    return tbl.search(embs.tolist()[0]).limit(3).to_pandas()

In [10]:
len(tbl)

100

In [11]:
# MobileSAM paper abstract 2nd half
query = """
Many of such applications need to be run on resource-constraint edge devices,
like mobile phones. In this work, we aim to make SAM mobile-friendly by replacing the heavyweight
image encoder with a lightweight one. A naive way to train such a new SAM as in the original SAM
paper leads to unsatisfactory performance, especially when limited training sources are available. We
find that this is mainly caused by the coupled optimization of the image encoder and mask decoder,
motivated by which we propose decoupled distillation. Concretely, we distill the knowledge from
the heavy image encoder (ViT-H in the original SAM) to a lightweight image encoder, which can be
automatically compatible with the mask decoder in the original SAM. The training can be completed
on a single GPU within less than one day, and the resulting lightweight SAM is termed MobileSAM
which is more than 60 times smaller yet performs on par with the original SAM. For inference speed,
With a single GPU, MobileSAM runs around 10ms per image: 8ms on the image encoder and 4ms
on the mask decoder. With superior performance, our MobileSAM is around 5 times faster than the
concurrent FastSAM and 7 times smaller, making it more suitable for mobile applications. Moreover,
we show that MobileSAM can run relatively smoothly on CPU
"""

result = search_table(query)

result.pop("vector")
display(HTML(result.to_html()))

,title,summary,authors,url,_distance
0,XFlow: Cross-modal Deep Neural Networks for Audiovisual Classification,"In recent years, there have been numerous developments towards solving\nmultimodal tasks, aiming to learn a stronger representation than through a\nsingle modality. Certain aspects of the data can be particularly useful in this\ncase - for example, correlations in the space or time domain across modalities\n- but should be wisely exploited in order to benefit from their full predictive\npotential. We propose two deep learning architectures with multimodal\ncross-connections that allow for dataflow between several feature extractors\n(XFlow). Our models derive more interpretable features and achieve better\nperformances than models which do not exchange representations, usefully\nexploiting correlations between audio and visual data, which have a different\ndimensionality and are nontrivially exchangeable. Our work improves on existing\nmultimodal deep learning algorithms in two essential ways: (1) it presents a\nnovel method for performing cross-modality (before features are learned from\nindividual modalities) and (2) extends the previously proposed\ncross-connections which only transfer information between streams that process\ncompatible data. Illustrating some of the representations learned by the\nconnections, we analyse their contribution to the increase in discrimination\nability and reveal their compatibility with a lip-reading network intermediate\nrepresentation. We provide the research community with Digits, a new dataset\nconsisting of three data types extracted from videos of people saying the\ndigits 0-9. Results show that both cross-modal architectures outperform their\nbaselines (by up to 11.5%) when evaluated on the AVletters, CUAVE and Digits\ndatasets, achieving state-of-the-art results.","[arxiv.Result.Author('Cătălina Cangea'), arxiv.Result.Author('Petar Veličković'), arxiv.Result.Author('Pietro Liò')]",http://arxiv.org/abs/1709.00572v2,40.346901
1,Dualing GANs,"Generative adversarial nets (GANs) are a promising technique for modeling a\ndistribution from samples. It is however well known that GAN training suffers\nfrom instability due to the nature of its maximin formulation. In this paper,\nwe explore ways to tackle the instability problem by dualizing the\ndiscriminator. We start from linear discriminators in which case conjugate\nduality provides a mechanism to reformulate the saddle point objective into a\nmaximization problem, such that both the generator and the discriminator of\nthis 'dualing GAN' act in concert. We then demonstrate how to extend this\nintuition to non-linear formulations. For GANs with linear discriminators our\napproach is able to remove the instability in training, while for GANs with\nnonlinear discriminators our approach provides an alternative to the commonly\nused GAN training algorithm.","[arxiv.Result.Author('Yujia Li'), arxiv.Result.Author('Alexander Schwing'), arxiv.Result.Author('Kuan-Chieh Wang'), arxiv.Result.Author('Richard Zemel')]",http://arxiv.org/abs/1706.06216v1,40.449284
2,Domain Generalization for Object Recognition with Multi-task Autoencoders,"The problem of domain generalization is to take knowledge acquired from a\nnumber of related domains where training data is available, and to then\nsuccessfully apply it to previously unseen domains. We propose a new feature\nlearning algorithm, Multi-Task Autoencoder (MTAE), that provides good\ngeneralization performance for cross-domain object recognition.\n Our algorithm extends the standard denoising autoencoder framework by\nsubstituting artificially induced corruption with naturally occurring\ninter-domain variability in the appearance of objects. Instead of\nreconstructing images from noisy versions, MTAE learns to transform the\noriginal image into analogs in multiple related domains. It thereby learns\nfeatures that are robust to variations across domains. The learnt features are\nthen used as inputs 

In [12]:
# Exmaple 2: Search via a concept you're reading
query = """
What is the general idea behind self-supervised learning.
"""

result = search_table(query)

result.pop("vector")
display(HTML(result.to_html()))

,title,summary,authors,url,_distance
0,A General Theory for Training Learning Machine,"Though the deep learning is pushing the machine learning to a new stage,\nbasic theories of machine learning are still limited. The principle of\nlearning, the role of the a prior knowledge, the role of neuron bias, and the\nbasis for choosing neural transfer function and cost function, etc., are still\nfar from clear. In this paper, we present a general theoretical framework for\nmachine learning. We classify the prior knowledge into common and\nproblem-dependent parts, and consider that the aim of learning is to maximally\nincorporate them. The principle we suggested for maximizing the former is the\ndesign risk minimization principle, while the neural transfer function, the\ncost function, as well as pretreatment of samples, are endowed with the role\nfor maximizing the latter. The role of the neuron bias is explained from a\ndifferent angle. We develop a Monte Carlo algorithm to establish the\ninput-output responses, and we control the input-output sensitivity of a\nlearning machine by controlling that of individual neurons. Applications of\nfunction approaching and smoothing, pattern recognition and classification, are\nprovided to illustrate how to train general learning machines based on our\ntheory and algorithm. Our method may in addition induce new applications, such\nas the transductive inference.",[arxiv.Result.Author('Hong Zhao')],http://arxiv.org/abs/1704.06885v1,33.708359
1,Learning Visual Reasoning Without Strong Priors,"Achieving artificial visual reasoning - the ability to answer image-related\nquestions which require a multi-step, high-level process - is an important step\ntowards artificial general intelligence. This multi-modal task requires\nlearning a question-dependent, structured reasoning process over images from\nlanguage. Standard deep learning approaches tend to exploit biases in the data\nrather than learn this underlying structure, while leading methods learn to\nvisually reason successfully but are hand-crafted for reasoning. We show that a\ngeneral-purpose, Conditional Batch Normalization approach achieves\nstate-of-the-art results on the CLEVR Visual Reasoning benchmark with a 2.4%\nerror rate. We outperform the next best end-to-end method (4.5%) and even\nmethods that use extra supervision (3.1%). We probe our model to shed light on\nhow it reasons, showing it has learned a question-dependent, multi-step\nprocess. Previous work has operated under the assumption that visual reasoning\ncalls for a specialized architecture, but we show that a general architecture\nwith proper conditioning can learn to visually reason effectively.","[arxiv.Result.Author('Ethan Perez'), arxiv.Result.Author('Harm de Vries'), arxiv.Result.Author('Florian Strub'), arxiv.Result.Author('Vincent Dumoulin'), arxiv.Result.Author('Aaron Courville')]",http://arxiv.org/abs/1707.03017v5,36.282284
2,Encoder Based Lifelong Learning,"This paper introduces a new lifelong learning solution where a single model\nis trained for a sequence of tasks. The main challenge that vision systems face\nin this context is catastrophic forgetting: as they tend to adapt to the most\nrecently seen task, they lose performance on the tasks that were learned\npreviously. Our method aims at preserving the knowledge of the previous tasks\nwhile learning a new one by using autoencoders. For each task, an\nunder-complete autoencoder is learned, capturing the features that are crucial\nfor its achievement. When a new task is presented to the system, we prevent the\nreconstructions of the features with these autoencoders from changing, which\nhas the effect of preserving the information on which the previous tasks are\nmainly relying. At the same time, the features are given space to adjust to the\nmost recent environment as only their projection into a low dimension\nsubmanifold is controlled. The proposed system is evaluated on image\nclassification tasks and shows a 

# Full Text Search
In text retrieval, full-text search refers to techniques for searching a single computer-stored document or a collection in a full-text database. Full-text search is distinguished from searches based on metadata or on parts of the original texts represented in databases

LanceDB now provides **experimental** support for full text search. This is currently Python only. We plan to push the integration down to Rust in the future to make this available for JS as well.


In [16]:
!pip install tantivy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 17.2 MB/s eta 0:00:00


### Build FTS index for the summary
Here, we're building the FTS index using python bindings for tantivy. You can also build the index for any other text column. A full-text index stores information about significant words and their location within one or more columns of a database table

In [17]:
# This cell might take a few mins
tbl.create_fts_index("summary")

In [18]:
## FTS via title
result = (
    tbl.search("What is the general idea behind self-supervised learning.")
    .limit(10)
    .to_pandas()
)

result.pop("vector")

display(HTML(result.to_html()))

,title,summary,authors,url,score
0,Expert Gate: Lifelong Learning with a Network of Experts,"In this paper we introduce a model of lifelong learning, based on a Network\nof Experts. New tasks / experts are learned and added to the model\nsequentially, building on what was learned before. To ensure scalability of\nthis process,data from previous tasks cannot be stored and hence is not\navailable when learning a new task. A critical issue in such context, not\naddressed in the literature so far, relates to the decision which expert to\ndeploy at test time. We introduce a set of gating autoencoders that learn a\nrepresentation for the task at hand, and, at test time, automatically forward\nthe test sample to the relevant expert. This also brings memory efficiency as\nonly one expert network has to be loaded into memory at any given time.\nFurther, the autoencoders inherently capture the relatedness of one task to\nanother, based on which the most relevant prior model to be used for training a\nnew expert, with finetuning or learning without-forgetting, can be selected. We\nevaluate our method on image classification and video prediction problems.","[arxiv.Result.Author('Rahaf Aljundi'), arxiv.Result.Author('Punarjay Chakravarty'), arxiv.Result.Author('Tinne Tuytelaars')]",http://arxiv.org/abs/1611.06194v2,4.703215
1,Approximate Bayesian Image Interpretation using Generative Probabilistic Graphics Programs,"The idea of computer vision as the Bayesian inverse problem to computer\ngraphics has a long history and an appealing elegance, but it has proved\ndifficult to directly implement. Instead, most vision tasks are approached via\ncomplex bottom-up processing pipelines. Here we show that it is possible to\nwrite short, simple probabilistic graphics programs that define flexible\ngenerative models and to automatically invert them to interpret real-world\nimages. Generative probabilistic graphics programs consist of a stochastic\nscene generator, a renderer based on graphics software, a stochastic likelihood\nmodel linking the renderer's output and the data, and latent variables that\nadjust the fidelity of the renderer and the tolerance of the likelihood model.\nRepresentations and algorithms from computer graphics, originally designed to\nproduce high-quality images, are instead used as the deterministic backbone for\nhighly approximate and stochastic generative models. This formulation combines\nprobabilistic programming, computer graphics, and approximate Bayesian\ncomputation, and depends only on general-purpose, automatic inference\ntechniques. We describe two applications: reading sequences of degraded and\nadversarially obscured alphanumeric characters, and inferring 3D road models\nfrom vehicle-mounted camera images. Each of the probabilistic graphics programs\nwe present relies on under 20 lines of probabilistic code, and supports\naccurate, approximately Bayesian inferences about ambiguous real-world images.","[arxiv.Result.Author('Vikash K. Mansinghka'), arxiv.Result.Author('Tejas D. Kulkarni'), arxiv.Result.Author('Yura N. Perov'), arxiv.Result.Author('Joshua B. Tenenbaum')]",http://arxiv.org/abs/1307.0060v1,4.515473
2,Learning Visual Reasoning Without Strong Priors,"Achieving artificial visual reasoning - the ability to answer image-related\nquestions which require a multi-step, high-level process - is an important step\ntowards artificial general intelligence. This multi-modal task requires\nlearning a question-dependent, structured reasoning process over images from\nlanguage. Standard deep learning approaches tend to exploit biases in the data\nrather than learn this underlying structure, while leading methods learn to\nvisually reason successfully but are hand-crafted for reasoning. We show that a\ngeneral-purpose, Conditional Batch Normalization approach achieves\nstate-of-the-art results on the CLEVR Visual Reasoning benchmark with a 2.4%\nerror rate. We outperform the next best end-to-end method (4.5%) and even\n

### Analysing OpenCLIP embeddings on Nomic
Atlas is a platform for interacting with both small and internet scale unstructured datasets.

Atlas enables you to:
* Store, update and organize multi-million point datasets of unstructured text, images and embeddings.
* Visually interact with embeddings of your data from a web browser.
* Operate over unstructured data and embeddings with topic modeling, semantic duplicate clustering and semantic search.
* Generate high dimensional and two-dimensional embeddings of your data.

In [19]:
!pip install nomic --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.1 MB/s eta 0:00:00


### Nomic Login

We are using Nomic to use Atlas for visualizing dataset in clusters

In [20]:
!nomic login

                                  Authenticate with the Nomic API                                   
                                  https://atlas.nomic.ai/cli-login                                  
       Click the above link to retrieve your access token and then run `nomic login [token]`        


In [22]:
!nomic login [token] # Paste your token from Nomic Ai cli login -- here

In [25]:
from nomic import atlas
import numpy as np

# Get pandas dataframe from lancedb table
df = tbl.to_pandas()

# get embeddings from df
embs = np.array(df.pop("vector").to_list())

project = atlas.map_data(embeddings=embs, data=df.to_dict("records"))
print()

2024-02-25 06:18:17.433 | WARNING  | nomic.atlas:map_data:96 - An ID field was not specified in your data so one was generated for you in insertion order.
2024-02-25 06:18:19.434 | INFO     | nomic.dataset:_create_project:868 - Creating dataset `inquisitive-jaynes`
2024-02-25 06:18:19.719 | INFO     | nomic.atlas:map_data:108 - Uploading data to Atlas.
1it [00:00,  1.62it/s]
2024-02-25 06:18:20.370 | INFO     | nomic.dataset:_add_data:1536 - Upload succeeded.
2024-02-25 06:18:20.374 | INFO     | nomic.atlas:map_data:123 - `prasantdixit9876/inquisitive-jaynes`: Data upload succeeded to dataset`
2024-02-25 06:18:20.655 | WARNING  | nomic.dataset:create_index:1121 - You did not specify the `topic_label_field` option in your topic_model, your dataset will not contain auto-labeled topics.
2024-02-25 06:18:21.396 | INFO     | nomic.dataset:create_index:1245 - Created map `inquisitive-jaynes` in dataset `prasantdixit9876/inquisitive-jaynes`: https://atlas.nomic.ai/data/prasantdixit9876/inquis

The visualizations are very interesting and is worth exploring more. In preliminary analysis, you can see that it succesfully creates clusters of similar types of papers. There are a few things that can be done next like comparing embeddings on various openclip models sizes and datasets.
<img width="1433" alt="Screenshot 2023-08-24 at 3 47 51 PM" src="https://github.com/lancedb/vectordb-recipes/assets/15766192/34ef88a3-2925-4450-abcd-1abc350ef3e4">